<a href="https://colab.research.google.com/github/iannjari/US-Flight-Delay-and-Cancellation/blob/main/src/explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import tarfile
from os import path

In [2]:
# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)


151681776

In [3]:
# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()


In [4]:
# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

True

In [5]:
import pandas as pd 

In [6]:
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})

In [7]:
df.shape

(2000000, 109)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 109 entries, Year to Div5TailNum
dtypes: float64(72), int64(18), object(19)
memory usage: 1.6+ GB


NameError: ignored

In [11]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DOT_ID_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginState',
 'DestAirportID',
 'DestState',
 'DepTime',
 'Cancelled']

In [12]:
featuredata=df[featurecols]
featuredata=featuredata.dropna()
featuredata.to_csv('featuredata.csv',index=False)

In [13]:
featuredata=pd.read_csv('featuredata.csv')
featuredata

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,Cancelled
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962965,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0,0.0
1962966,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0,0.0
1962967,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0,0.0
1962968,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0,0.0


In [14]:
target_name = "Cancelled"
target = featuredata[target_name]
data = featuredata.drop(columns=[target_name])

In [15]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

In [16]:
data.isnull().values.any()

False

In [17]:
data

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1962965,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0
1962966,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0
1962967,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0
1962968,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0


In [18]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(data, target)
dummy_preds=dummy_clf.predict(data)
dummy_clf.predict(data)
dummy_clf.score(data, target)

0.9997671895138489

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, data, target, cv=10,return_estimator=True)

cv_results

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'estimator': [Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticReg

In [35]:
array=cv_results["test_score"]
array.mean()

0.9969363260723176

In [21]:
logistic_regression.fit(data,target)
predictions=logistic_regression.predict(data.iloc[0:1])
predictions

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


array([0.])

In [22]:
data.iloc[0:1]

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0


In [23]:
predictions=logistic_regression.predict(data)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [24]:
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(target, predictions)
c_matrix

array([[1962513,       0],
       [    457,       0]])

In [25]:
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(1962513, 0, 457, 0)

In [26]:
precision= tp/(tp+fp)
precision

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in long_scalars
  """Entry point for launching an IPython kernel.


nan

In [27]:
c_matrix_dummy=confusion_matrix(target, dummy_preds)
tn, fp, fn, tp = c_matrix_dummy.ravel()
(tn, fp, fn, tp)

(1962513, 0, 457, 0)

In [28]:
c_matrix_=confusion_matrix(predictions, dummy_preds)
c_matrix_


array([[1962970]])

In [29]:
y_true = [1,1,1,1,1,1,1]
y_pred = [1,1,1,1,1,1,1]
confusion_matrix(y_true, y_pred)

array([[7]])

In [30]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

Class imbalance is an issue


In [31]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

# fit predictor and target 

x_ros, y_ros = ros.fit_resample(data, target)

y_ros=pd.DataFrame(y_ros)

print('Original dataset shape', target.value_counts())
print('Resample dataset shape', y_ros.value_counts())

Original dataset shape 0.0    1962513
1.0        457
Name: Cancelled, dtype: int64
Resample dataset shape Cancelled
1.0          1962513
0.0          1962513
dtype: int64


In [32]:
y_ros

,Cancelled
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3925021,1.0
3925022,1.0
3925023,1.0
3925024,1.0


In [34]:
logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


cv_results = cross_validate(
    logistic_regression, x_ros, y_ros, cv=10,return_estimator=True)

cv_results

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

{'estimator': [Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticRegression())]),
  Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                  ('logisticregression', LogisticReg

In [36]:
array=cv_results["test_score"]
array.mean()

0.9969363260723176

In [38]:
logistic_regression.fit(x_ros,y_ros)
predictions=logistic_regression.predict(x_ros)
c_matrix=confusion_matrix(y_ros, predictions)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: ignored

In [ ]:
precision= tp/(tp+fp)
precision

In [ ]:
from sklearn.ensemble import (RandomForestClassifier, StackingClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from  sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df=pd.DataFrame(x_ros)
df

In [ ]:
df['target']=y_ros
df

In [ ]:
df2=df.sample(n=300000)
df2['target'].value_counts()

In [ ]:
x,y = df2.drop(columns=['target']),df2['target']
x_train, x_test, y_train, y_test = train_test_split(x,y,stratify=y,random_state=42)

In [ ]:
estimators=[
            ("rf",RandomForestClassifier(n_estimators=10,random_state=42)),
            ("gb",GradientBoostingClassifier(n_estimators=10,random_state=42)),
            ("knn",KNeighborsClassifier(n_neighbors=5))
]

In [ ]:
clf=StackingClassifier(estimators=estimators,final_estimator=LogisticRegression())

In [ ]:
pipeline = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), clf
)

pipeline.fit(x_train,y_train)
pipeline.score(x_test,y_test)

In [ ]:
y[:100]

In [ ]:
preds=pipeline.predict(x[:10000])
c_matrix=confusion_matrix(y[:10000], preds)
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

In [ ]:
precision= tp/(tp+fp)
precision

In [ ]:
accuracy=(tp+tn)/(tp+tn+fn+fp)
accuracy